This module performs simulation with BESS. The simulation process makes decision of the BESS output at each time step.

In [1]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from math import *

from settings import *
from lo_step import lo_step

#### Load data and extract time series of solar power and building load

In [3]:
path_data = os.path.join("data", "sample_data.csv")
data = pd.read_csv(path_data)

dts = np.array(data['datetime'])
PSs = np.array(data['solar_power'])
LBs = np.array(data['building_load'])

#### Initialize parameters

In [10]:
N = len(dts)
Ppeak = 0 # Initialize peak power
SoC = 0.5 # Initialize SoC of the BESS
SoCs = [SoC] # Initialize SoC time series
BESS_outputs = [] # Initialize BESS time series
Qs = []

#### Running the simulation over each time step.
At each time step, make a decision on the BESS output

In [11]:
for i in range(N):
    dt = datetime.strptime(dts[i], "%Y%m%d-%H%M%S")
    
    # Determine weekday
    if dt.weekday() in [0, 1, 2, 3, 4]:
        # Determine month
        if dt.month in [3, 4]:
            peakHourTable = peakHourTable_weekdayMarApr
        else:
            peakHourTable = peakHourTable_weekday
    else:
        peakHourTable = peakHourTable_weekend
    
    # Determine season
    if dt.month in summerMonths:
        energyChargeRef = energyChargeRef_Summer
        demandChargeRef = demandChargeRef_Summer
    else:
        energyChargeRef = energyChargeRef_Winter
        demandChargeRef = demandChargeRef_Winter

    # Get energy charge and demand charge at t
    peak_indicator = peakHourTable[dt.hour]
    pE = energyChargeRef[peak_indicator]
    pF = pE # net-metering
    pD = demandChargeRef[peak_indicator]
    V = VsTable[peak_indicator]
    
    # Get BESS output
    LB = LBs[i]
    PS = PSs[i]
    x = lo_step(LB, PS, Ppeak, SoC, pE, pF, pD, V, DT)
    BESS_outputs.append(x)
    
    # Update SoC, Ppeak, and !
    E = SoC*E_FULL - x*DT - abs(x)*DT*(1-sqrt(K))
    SoC = E/E_FULL
    SoCs.append(SoC)
    Ppeak = max(Ppeak, LB-PS-x)
    
    Q = E_MAX - SoC*E_FULL - DT*P_MAX*sqrt(K)
    Qs.append(Q)
    
    if i % 10000 == 0:
        print(dt)
        
print("Finished")

2022-05-31 00:00:00
2022-05-31 02:47:52
2022-05-31 05:35:35
2022-05-31 08:23:16
2022-05-31 11:11:10
2022-05-31 13:59:04
2022-05-31 16:46:50
2022-05-31 19:34:45
2022-05-31 22:22:26
2022-06-01 01:28:03
2022-06-01 04:15:55
2022-06-01 07:03:47
2022-06-01 09:51:35
2022-06-01 12:39:22
2022-06-01 15:27:11
2022-06-01 18:15:08
2022-06-01 21:03:01
2022-06-01 23:50:57
Finished


#### Output results

In [13]:
info_export = {'datetime': dts,\
               'solar_power': PSs,\
               'building_load': LBs,\
               'BESS_output': BESS_outputs,
               'SoC': SoCs[1:]}
df = pd.DataFrame(info_export)
path_res = os.path.join("results", f"load_with_BESS.csv")
df.to_csv(path_res)
df

,datetime,solar_power,building_load,BESS_output,SoC
0,20220531-000000,0.0,22.838876,0.000000,0.500000
1,20220531-000001,0.0,24.012708,0.000000,0.500000
2,20220531-000002,0.0,23.201554,0.000000,0.500000
3,20220531-000003,0.0,22.433628,0.000000,0.500000
4,20220531-000004,0.0,22.718627,0.000000,0.500000
...,...,...,...,...,...
170534,20220601-235955,0.0,29.653365,4.653365,0.729848
170535,20220601-235956,0.0,29.575575,4.575575,0.729844
170536,20220601-235957,0.0,29.751220,4.751220,0.729840
170537,20220601-235958,0.0,29.613346,4.613346,0.729836
